<a href="https://colab.research.google.com/github/ysangchoi/choi-young-sang/blob/master/deepfake_practice_0829_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notice

본 notebook은 first-order-motion의 데모를 쉽게 만들고 음성까지 씌울 수 있도록 제작한 노트북입니다.   

-작성자 박결-

#1. 준비

In [ ]:
# 1.구글 드라이브 연결
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%pwd

'/content'

In [ ]:
%cd gdrive/

/content/gdrive


In [ ]:
%whos

Variable   Type      Data/Info
------------------------------
drive      module    <module 'google.colab.dri<...>s/google/colab/drive.py'>


In [ ]:
# 2.실행할 경로로 이동

import os
if os.path.exists('/content/gdrive/My Drive/FOMM') is False:
  os.mkdir('/content/gdrive/My Drive/FOMM')
%cd /content/gdrive/My Drive/FOMM


/content/gdrive/My Drive/FOMM


In [ ]:
!dir

first-order-model  vox-adv-cpk.pth.tar


In [ ]:

# 3.소스코드 다운로드
if os.path.exists('/content/gdrive/My Drive/FOMM/first-order-model') is False:
  !git clone https://github.com/AliaksandrSiarohin/first-order-model


In [ ]:

# 4.소스 폴더로 이동
%cd first-order-model
if os.path.exists('my_data') is False:
  os.mkdir('my_data')

global path
path ='my_data'   # 데이터가 있는 폴더

print('정상 실행 완료')

/content/gdrive/My Drive/FOMM/first-order-model
정상 실행 완료


In [ ]:
%pwd

'/content/gdrive/MyDrive/FOMM/first-order-model'

# ※파일 올리기

## 이 곳이 사용자가 설정할 유일한 섹션입니다. 아래 순서대로 진행하세요.
1. 사진과 동영상 파일은 __My Drive/FOMM/first-order-model__ 폴더 안에 있는 __'my_data'__에 넣어주세요.  
※입력 데이터(사진과 동영상)의 비율은 1:1이어야 합니다. 사진과 동영상을 편집 툴을 사용해 얼굴 위치를 중심으로 1:1 비율로 잘라주세요.  

2. __'my_data'__에 넣은 파일 이름을 아래 셀의 __image__와 __video__에 적어주세요.
3. 최종 완성한 동영상의 이름을 __result_name__에 적어주세요. 

In [ ]:
#이미지와 비디오 이름을 적으세요.
#image = 'ains.jpg' # 불러올 이미지 이름
#image = 'image-0829.jpg'
#video = 'luna.mp4' # 불러올 비디오 이름
image = 'man-0829.jpg'
video = 'test-0829.mp4' # 불러올 비디오 이름
#저장할 파일명을 적으세요.
result_name = '최종.mp4' #저장할 파일 이름 



#학습한 체크포인트 불러오기

### 아래 URL에서 __vox-adv-ckp.pth.tar__ 또는 __vox-ckp.pth.tar__파일을 사본으로 복사하여 first-order-model 폴더에 넣으면 됩니다.
  

https://drive.google.com/drive/folders/1PyQJmkdCsAkOYwUyaj_l-l0as-iLDgeH

#### 체크포인트 파일은 드라이브에 저장 후 보존됩니다.


In [ ]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from demo import load_checkpoints
#설정파일과 checkpoint파일 경로
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='vox-adv-cpk.pth.tar') #다운받은 checkpoint파일 경로를 지정

#만약 FileNotFoundError가 발생한다면, 본 노트북을 '런타임초기화' 후 다시 시작하세요.

/content/gdrive/MyDrive/FOMM/first-order-model/demo.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# 2.필요한 라이브러리 불러오기

In [ ]:
%whos

Variable           Type                        Data/Info
--------------------------------------------------------
drive              module                      <module 'google.colab.dri<...>s/google/colab/drive.py'>
generator          DataParallelWithCallback    DataParallelWithCallback(<...>, padding=(3, 3))\n  )\n)
image              str                         man-0829.jpg
kp_detector        DataParallelWithCallback    DataParallelWithCallback(<...>Interpolation2d()\n  )\n)
load_checkpoints   function                    <function load_checkpoints at 0x7f3d57e83d40>
os                 module                      <module 'os' from '/usr/lib/python3.7/os.py'>
path               str                         my_data
result_name        str                         최종.mp4
video              str                         test-0829.mp4


In [ ]:
# !pip install imageio-ffmpeg
!pip3 install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import imageio
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
from demo import make_animation
from skimage import img_as_ubyte

import moviepy.editor as mp
from moviepy.editor import *
!pip install -U kora
from kora.drive import upload_public


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)4014080/45929032 bytes (8.7%)8036352/45929032 bytes (17.5%)10477568/45929032 bytes (22.8%)13385728/45929032 bytes (29.1%)16785408/45929032 bytes (36.5%)20668416/45929032 bytes (45.0%)24805376/45929032 bytes (54.0%)28672000/45929032 bytes (62.4%)32694272/45929032 bytes (71.2%)36593664/45929032 bytes (79.7%)40493056/45929032 bytes (88.2%)44605440/45929032 bytes (97.1%)

In [ ]:
%whos

Variable                 Type                        Data/Info
--------------------------------------------------------------
AudioClip                type                        <class 'moviepy.audio.AudioClip.AudioClip'>
AudioFileClip            type                        <class 'moviepy.audio.io.<...>oFileClip.AudioFileClip'>
ColorClip                type                        <class 'moviepy.video.VideoClip.ColorClip'>
CompositeAudioClip       type                        <class 'moviepy.audio.Aud<...>Clip.CompositeAudioClip'>
CompositeVideoClip       type                        <class 'moviepy.video.com<...>Clip.CompositeVideoClip'>
HTML                     type                        <class 'IPython.core.display.HTML'>
ImageClip                type                        <class 'moviepy.video.VideoClip.ImageClip'>
ImageSequenceClip        type                        <class 'moviepy.video.io.<...>eClip.ImageSequenceClip'>
TextClip                 type                        <clas

# 3.웹페이지 화면에 디스플레이할수있는 함수 정의

In [ ]:
#화면을 보여줄 디스플레이 함수

def display( driving, source=None, generated=None, fps = None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = []
        if source is not None:
          cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
    if fps is None:
      ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    else:
      fps = 1000. / fps
      ani = animation.ArtistAnimation(fig, ims, interval=fps, repeat_delay=1000)
    plt.close()
    return ani

# 4. Data 불러오기 

In [ ]:
#8-2
#이미지와 비디오 불러오기

image_path = os.path.join(path, image)
video_path = os.path.join(path, video)

source_image = imageio.imread(image_path)
driving_video = imageio.get_reader(video_path)

#5. 동영상 FPS 확인


In [ ]:
fps = driving_video.get_meta_data()['fps']
fps

30.0

# 6.prediction

In [ ]:
# 전처리
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

In [ ]:
#결과물 생성
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

100%|██████████| 483/483 [00:21<00:00, 22.98it/s]


#7. 결과물 확인

In [ ]:
#함수에 데이터를 넣고 비디오를 만듬.
result_video = display(driving = driving_video, 
                       source = None,    #이미지를 원하지 않을 경우 None 설정
                       generated = predictions,  #prediction 하지 않은 경우 None 설정
                       fps=fps)

In [ ]:
HTML(result_video.to_html5_video())

#8. 비디오 저장


In [ ]:
result_path = os.path.join('my_data/', result_name)

imageio.mimsave(result_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)
# result_video.save(result_path)

#9. 결과물에 음성 씌우기(함수 정의)

In [ ]:
# 함수 정의
def make_video(origin, gen, save):
  
  audio_path = os.path.join(path, 'audio.mp3')

  clip = mp.VideoFileClip(origin)
  clip.audio.write_audiofile(audio_path)

  videoclip = VideoFileClip(gen)
  audioclip = AudioFileClip(audio_path)
  new_audioclip = CompositeAudioClip([audioclip])
  videoclip.audio = new_audioclip
  videoclip.write_videofile(save)
  !rm -f {audio_path}

# 10. 음성 비디오 저장

In [ ]:
#최종 결과물 이름
save_name = result_name.split('.')[0] +'(with_voice).mp4' 

#음성 비디오 생성과 저장
save_path = os.path.join(path, save_name)
make_video(video_path, result_path, save_path)

[MoviePy] Writing audio in my_data/audio.mp3


100%|██████████| 357/357 [00:00<00:00, 636.84it/s]

[MoviePy] Done.


[MoviePy] >>>> Building video my_data/최종(with_voice).mp4
[MoviePy] Writing audio in 최종(with_voice)TEMP_MPY_wvf_snd.mp3


100%|██████████| 358/358 [00:00<00:00, 637.00it/s]

[MoviePy] Done.
[MoviePy] Writing video my_data/최종(with_voice).mp4



100%|██████████| 484/484 [00:02<00:00, 172.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: my_data/최종(with_voice).mp4 



#11. 최종 결과물 보이기

In [ ]:
url = upload_public(save_path)
# then display it
HTML(f"""<video src={url} height=300 controls/>""")
